Install Dependencies

In [ ]:
! python -m nltk.downloader 'punkt' 
! python -m nltk.downloader 'stopwrods'
! python -m nltk.downloader 'wordnet'

In [2]:
base_data_files_path = '/Users/shahriar/Downloads/arxiv dataset'
arxiv_10000_path = f'{base_data_files_path}/arxiv_10000.json'
arxiv_complete_path = f'{base_data_files_path}/arxiv-metadata-oai-snapshot.json'
temp_dir = 'tmp/'
test_articles_dir = 'test_articles/'
articles_to_plagiarise_dir = 'articles_to_plagiarise/'
test_metadata_path = f'test_metadata/metadata.json'
exact_copies_count = 50
exact_copies_range = (0, 50)
near_copies_count = 50
near_copies_range = (50, 100)
substitutes_count = 300
substitutes_range= (100, 400)
paraphrases_count = 5
paraphrases_range = (400, 410)
base_test_article_id = '0706.0638'
base_article_url = 'https://arxiv.org/pdf'

searchable_fields = ['title', 'categories', 'abstract']

articles_count = 10000

elastic_data = {
    'indices': {
        'articles': {
            'settings': {
                'index.number_of_replicas': 0,
                'index.number_of_shards': 1,
            },
            'mappings': {
                'properties': {
                    'id': {"type": "keyword"},
                    'authors': {'type': 'keyword'},
                    'last_modified': {'type': 'date'},
                    'title': {'type': 'text'},
                    'original_title': {'type': 'text'},
                    'doi': {'type': 'keyword'},
                    'categories': {'type': 'text'},
                    'abstract': {'type': 'text'},
                    'original_abstract': {'type': 'text'},
                }
            }
        },
    }
}

category_map = {
    'astro-ph': 'Astrophysics',
    'astro-ph.CO': 'Cosmology and Nongalactic Astrophysics',
    'astro-ph.EP': 'Earth and Planetary Astrophysics',
    'astro-ph.GA': 'Astrophysics of Galaxies',
    'astro-ph.HE': 'High Energy Astrophysical Phenomena',
    'astro-ph.IM': 'Instrumentation and Methods for Astrophysics',
    'astro-ph.SR': 'Solar and Stellar Astrophysics',
    'cond-mat.dis-nn': 'Disordered Systems and Neural Networks',
    'cond-mat.mes-hall': 'Mesoscale and Nanoscale Physics',
    'cond-mat.mtrl-sci': 'Materials Science',
    'cond-mat.other': 'Other Condensed Matter',
    'cond-mat.quant-gas': 'Quantum Gases',
    'cond-mat.soft': 'Soft Condensed Matter',
    'cond-mat.stat-mech': 'Statistical Mechanics',
    'cond-mat.str-el': 'Strongly Correlated Electrons',
    'cond-mat.supr-con': 'Superconductivity',
    'cs.AI': 'Artificial Intelligence',
    'cs.AR': 'Hardware Architecture',
    'cs.CC': 'Computational Complexity',
    'cs.CE': 'Computational Engineering, Finance, and Science',
    'cs.CG': 'Computational Geometry',
    'cs.CL': 'Computation and Language',
    'cs.CR': 'Cryptography and Security',
    'cs.CV': 'Computer Vision and Pattern Recognition',
    'cs.CY': 'Computers and Society',
    'cs.DB': 'Databases',
    'cs.DC': 'Distributed, Parallel, and Cluster Computing',
    'cs.DL': 'Digital Libraries',
    'cs.DM': 'Discrete Mathematics',
    'cs.DS': 'Data Structures and Algorithms',
    'cs.ET': 'Emerging Technologies',
    'cs.FL': 'Formal Languages and Automata Theory',
    'cs.GL': 'General Literature',
    'cs.GR': 'Graphics',
    'cs.GT': 'Computer Science and Game Theory',
    'cs.HC': 'Human-Computer Interaction',
    'cs.IR': 'Information Retrieval',
    'cs.IT': 'Information Theory',
    'cs.LG': 'Machine Learning',
    'cs.LO': 'Logic in Computer Science',
    'cs.MA': 'Multiagent Systems',
    'cs.MM': 'Multimedia',
    'cs.MS': 'Mathematical Software',
    'cs.NA': 'Numerical Analysis',
    'cs.NE': 'Neural and Evolutionary Computing',
    'cs.NI': 'Networking and Internet Architecture',
    'cs.OH': 'Other Computer Science',
    'cs.OS': 'Operating Systems',
    'cs.PF': 'Performance',
    'cs.PL': 'Programming Languages',
    'cs.RO': 'Robotics',
    'cs.SC': 'Symbolic Computation',
    'cs.SD': 'Sound',
    'cs.SE': 'Software Engineering',
    'cs.SI': 'Social and Information Networks',
    'cs.SY': 'Systems and Control',
    'econ.EM': 'Econometrics',
    'eess.AS': 'Audio and Speech Processing',
    'eess.IV': 'Image and Video Processing',
    'eess.SP': 'Signal Processing',
    'gr-qc': 'General Relativity and Quantum Cosmology',
    'hep-ex': 'High Energy Physics - Experiment',
    'hep-lat': 'High Energy Physics - Lattice',
    'hep-ph': 'High Energy Physics - Phenomenology',
    'hep-th': 'High Energy Physics - Theory',
    'math.AC': 'Commutative Algebra',
    'math.AG': 'Algebraic Geometry',
    'math.AP': 'Analysis of PDEs',
    'math.AT': 'Algebraic Topology',
    'math.CA': 'Classical Analysis and ODEs',
    'math.CO': 'Combinatorics',
    'math.CT': 'Category Theory',
    'math.CV': 'Complex Variables',
    'math.DG': 'Differential Geometry',
    'math.DS': 'Dynamical Systems',
    'math.FA': 'Functional Analysis',
    'math.GM': 'General Mathematics',
    'math.GN': 'General Topology',
    'math.GR': 'Group Theory',
    'math.GT': 'Geometric Topology',
    'math.HO': 'History and Overview',
    'math.IT': 'Information Theory',
    'math.KT': 'K-Theory and Homology',
    'math.LO': 'Logic',
    'math.MG': 'Metric Geometry',
    'math.MP': 'Mathematical Physics',
    'math.NA': 'Numerical Analysis',
    'math.NT': 'Number Theory',
    'math.OA': 'Operator Algebras',
    'math.OC': 'Optimization and Control',
    'math.PR': 'Probability',
    'math.QA': 'Quantum Algebra',
    'math.RA': 'Rings and Algebras',
    'math.RT': 'Representation Theory',
    'math.SG': 'Symplectic Geometry',
    'math.SP': 'Spectral Theory',
    'math.ST': 'Statistics Theory',
    'math-ph': 'Mathematical Physics',
    'nlin.AO': 'Adaptation and Self-Organizing Systems',
    'nlin.CD': 'Chaotic Dynamics',
    'nlin.CG': 'Cellular Automata and Lattice Gases',
    'nlin.PS': 'Pattern Formation and Solitons',
    'nlin.SI': 'Exactly Solvable and Integrable Systems',
    'nucl-ex': 'Nuclear Experiment',
    'nucl-th': 'Nuclear Theory',
    'physics.acc-ph': 'Accelerator Physics',
    'physics.ao-ph': 'Atmospheric and Oceanic Physics',
    'physics.app-ph': 'Applied Physics',
    'physics.atm-clus': 'Atomic and Molecular Clusters',
    'physics.atom-ph': 'Atomic Physics',
    'physics.bio-ph': 'Biological Physics',
    'physics.chem-ph': 'Chemical Physics',
    'physics.class-ph': 'Classical Physics',
    'physics.comp-ph': 'Computational Physics',
    'physics.data-an': 'Data Analysis, Statistics and Probability',
    'physics.ed-ph': 'Physics Education',
    'physics.flu-dyn': 'Fluid Dynamics',
    'physics.gen-ph': 'General Physics',
    'physics.geo-ph': 'Geophysics',
    'physics.hist-ph': 'History and Philosophy of Physics',
    'physics.ins-det': 'Instrumentation and Detectors',
    'physics.med-ph': 'Medical Physics',
    'physics.optics': 'Optics',
    'physics.plasm-ph': 'Plasma Physics',
    'physics.pop-ph': 'Popular Physics',
    'physics.soc-ph': 'Physics and Society',
    'physics.space-ph': 'Space Physics',
    'q-bio.BM': 'Biomolecules',
    'q-bio.CB': 'Cell Behavior',
    'q-bio.GN': 'Genomics',
    'q-bio.MN': 'Molecular Networks',
    'q-bio.NC': 'Neurons and Cognition',
    'q-bio.OT': 'Other Quantitative Biology',
    'q-bio.PE': 'Populations and Evolution',
    'q-bio.QM': 'Quantitative Methods',
    'q-bio.SC': 'Subcellular Processes',
    'q-bio.TO': 'Tissues and Organs',
    'q-fin.CP': 'Computational Finance',
    'q-fin.EC': 'Economics',
    'q-fin.GN': 'General Finance',
    'q-fin.MF': 'Mathematical Finance',
    'q-fin.PM': 'Portfolio Management',
    'q-fin.PR': 'Pricing of Securities',
    'q-fin.RM': 'Risk Management',
    'q-fin.ST': 'Statistical Finance',
    'q-fin.TR': 'Trading and Market Microstructure',
    'quant-ph': 'Quantum Physics',
    'stat.AP': 'Applications',
    'stat.CO': 'Computation',
    'stat.ME': 'Methodology',
    'stat.ML': 'Machine Learning',
    'stat.OT': 'Other Statistics',
    'stat.TH': 'Statistics Theory'
}

print_divider = '\n*************************************************************************\n'

Utility Functions

In [21]:
import requests
import pypdf
from io import BytesIO
import scipdf
import time
# import urllib
import os
from nltk.corpus import stopwords
import re
from unidecode import unidecode
import spacy
from spacy.lang.en.stop_words import STOP_WORDS #import commen list of stopword
import en_core_sci_lg  # import downlaoded model
import string
# Parser
parser = en_core_sci_lg.load()
parser.max_length = 7000000 #Limit the size of the parser
punctuations = string.punctuation #list of punctuation to remove from text
stop_words = stopwords.words('english')
stopwords = list(STOP_WORDS)

def preprocess(sentence):
    parsed_sentence = parser(sentence.replace('\n', ' '))
    tokens = [word.lemma_.lower().strip() 
                if word.lemma_ != "-PRON-" 
                else word.lower_ for word in parsed_sentence
             ] # transform to lowercase and then split the scentence
    return ' '.join([unidecode(word) for word in tokens 
                    if word not in stopwords and word not in punctuations
                    ]) #remove stopsword an punctuation

def convert_pdf_bytes_to_str(pdf_bytes):
    reader = pypdf.PdfReader(BytesIO(pdf_bytes))
    return ' '.join([page.extract_text() for page in reader.pages])

def clean_id(dir: str):
    return dir.replace('/', '_')

def get_article_text(article_id: str, folder: str):
    text = ''
    local_path = f'{folder}/{clean_id(article_id)}'
    if not os.path.isdir(folder):
        os.makedirs(folder)
    if os.path.isfile(local_path):
        with open(local_path, 'r') as file:
            text = '\n'.join(file.readlines())
    else:
        article_url = f'{base_article_url}/{article_id}'
        response = requests.get(article_url)
        text = convert_pdf_bytes_to_str(response.content)
        with open(local_path, 'w') as file:
            file.write(text)
    return text
    

In [5]:
from typing import Dict
from elasticsearch import Elasticsearch as ElasticsearchClient
from elasticsearch import NotFoundError


class EsConfig:
    def __init__(self, **kwargs):
        self.host: str = kwargs["host"]
        self.port: int = kwargs["port"]
        self.request_timeout: str = kwargs["request_timeout"]
        self.max_retries: int = kwargs["max_retries"]


class Elasticsearch:
    def __init__(self, config):
        self.config = config
        self.client = ElasticsearchClient(
                    hosts="http://" + self.config.host + ":" + self.config.port,
                    request_timeout=int(self.config.request_timeout[:-1]),
                    retry_on_timeout=True,
                    max_retries=self.config.max_retries,
                )
        self._create_or_update_indices()

    def _create_or_update_indices(self):
        for index_name in elastic_data['indices']:
            existing_indices = self._cat_indices(index_name)
            if not existing_indices or len(existing_indices) == 0:
                self._create_index(index_name)
                print(f"created articles index")
            else:
                self._update_index(index_name)
                print(f"articles index updated")


    def _cat_indices(self, name: str) -> Dict:
        response = []
        try:
            response = self.client.cat.indices(
                index=name,
                s="creation.date",
                h="index,docs.count,creation.date",
                format="json",
            )
        except NotFoundError:
            return []
        return response


    def _create_index(self, index_name):
        settings = elastic_data['indices'][index_name]['settings']
        mappings = elastic_data['indices'][index_name]['mappings']
        self.client.indices.create(index=index_name, settings=settings, mappings=mappings)

    def _update_index(self, index_name):
        mapping_properties = elastic_data['indices'][index_name]['mappings']['properties']
        self.client.indices.put_mapping(index=index_name, properties=mapping_properties)


    def index(self, index_name, document, id_):
        return self.client.index(
            index=index_name,
            document=document,
            id=id_,
            timeout=self.config.request_timeout,
        )

    def search(self, queries, n):
        query = {
                "bool": {
                    "should": [
                        {"match": {field: value}} for field, value in queries if value is not None and value != "" 
                    ]
                },
            }
        return self.client.search(
            query=query,
            size=n,
        )

    def get_article(self, article_id: str) -> Dict:
        return es.client.get(
                index='articles',
                id=article_id,
            )['_source']


es_config = EsConfig(host='localhost', port='9200', request_timeout= '3s', max_retries=10)
es = Elasticsearch(config=es_config)

articles index updated


Utility Functions

# Index Documents into Elasticsearch

In [129]:
import json

def populate_elasticsearch():
    with open(arxiv_complete_path, 'r') as file:
        for line in file.readlines():
            article = json.loads(line)
            doc = {
                'id': clean_id(article['id']),
                'authors': [' '.join(author_array) for author_array in article['authors_parsed']],
                'last_modified': article['update_date'],
                'title': preprocess(article['title']),
                'original_title': article['title'],
                'doi': article['doi'],
                'categories': article['categories'].split(' '),
                'abstract': preprocess(article['abstract']),
                'original_abstract': article['abstract'],
            }
            es.index(
                index_name='articles',
                document=doc,
                id_=doc['id'],
            )


Obfuscation

    0. Download 1000 random articles

In [11]:
from numpy import random

if False:
    for i in range(1):
        doc_number = random.randint(articles_count)
        doc_id = es.client.search(
            index='articles',
            query={'match_all':{}},
            from_=doc_number,
            size=1,
        )['hits']['hits'][0]['_source']['id']
        try:
            doc_text = get_article_text(doc_id, articles_to_plagiarise_dir)
        except:
            continue


Create Metadata

In [108]:
import os
import json

if True:

    metadata = {
        'exact_copies_ids': [],
        'near_copies_ids': [],
        'substitutes_ids': [],
        'paraphrases_ids': [],
    }
    for i, article_id in enumerate(os.listdir(articles_to_plagiarise_dir)):
        if i < exact_copies_count:
            metadata['exact_copies_ids'].append(article_id)
        elif i - exact_copies_count < near_copies_count:
            metadata['near_copies_ids'].append(article_id)
        elif i - exact_copies_count - near_copies_count < substitutes_count:
            metadata['substitutes_ids'].append(article_id)
        else:
            metadata['paraphrases_ids'].append(article_id)

    test_metadata_dir = ''.join(test_metadata_path.split('/')[:-1])
    if not os.path.isdir(test_metadata_dir):
        os.makedirs(test_metadata_dir)

    with open(test_metadata_path, 'w') as file:
        json.dump(metadata, file)


Load Metadata

In [6]:
import json
metadata = None
with open(test_metadata_path, 'r') as file:
    metadata = json.load(file)

Load Base Test Article

In [7]:
from nltk import sent_tokenize
base_test_article = {"id":"0706.0638","submitter":"Marc Wambst","authors":"Philippe Nuss (IRMA), Marc Wambst (IRMA)","title":"Non-Abelian Hopf Cohomology II -- the General Case --","comments":"This paper is conceived as the continuation of the article of the\n  same authors entitled \"Non-Abelian Hopf Cohomology\", J. Algebra t. 312,\n  (2007), no 2, p. 733 -- 754","journal-ref":"Journal of Algebra 319 (2008) 4621--4645","doi":None,"report-no":None,"categories":"math.QA","license":None,"abstract":"  We introduce and study non-abelian cohomology sets of Hopf algebras with\ncoefficients in Hopf comodule algebras. We prove that these sets generalize as\nwell Serre's non-abelian group cohomology theory as the cohomological theory\nconstructed by the authors in a previous article. We establish their\nfunctoriality and compute explicit examples. Further we classify Hopf torsors.\n","versions":[{"version":"v1","created":"Tue, 5 Jun 2007 12:19:11 GMT"}],"update_date":"2008-05-15","authors_parsed":[["Nuss","Philippe","","IRMA"],["Wambst","Marc","","IRMA"]]}
base_test_article = {
    'id': clean_id(base_test_article['id']),
    'authors': [' '.join(author_array) for author_array in base_test_article['authors_parsed']],
    'last_modified': base_test_article['update_date'],
    'title': preprocess(base_test_article['title']),
    'original_title': base_test_article['title'],
    'doi': base_test_article['doi'],
    'categories': base_test_article['categories'].split(' '),
    'abstract': preprocess(base_test_article['abstract']),
    'original_abstract': base_test_article['abstract'],
}
base_test_article_text = get_article_text(base_test_article_id, articles_to_plagiarise_dir)
base_title_tokenized = sent_tokenize(base_test_article['original_title'])
print(f'base title: {base_test_article["original_title"]}', end=print_divider)
print(f'base abstract: {base_test_article["original_abstract"]}')

base title: Non-Abelian Hopf Cohomology II -- the General Case --
*************************************************************************
base abstract:   We introduce and study non-abelian cohomology sets of Hopf algebras with
coefficients in Hopf comodule algebras. We prove that these sets generalize as
well Serre's non-abelian group cohomology theory as the cohomological theory
constructed by the authors in a previous article. We establish their
functoriality and compute explicit examples. Further we classify Hopf torsors.



    1. Exact Copy + rearrangement

In [40]:
from nltk.tokenize import sent_tokenize, word_tokenize
from numpy import random
import re

def simple_exact_text_copier(original_text: str, base_text: str, max_copy_count: int = None, max_copy_percentage: int = None):
    tmp = sent_tokenize(re.sub('[\n ]+', ' ', original_text))
    org_text_tokenized = [sentence for sentence in tmp if len(word_tokenize(sentence.strip())) > 10] or [tmp[0]]
    new_text_tokenized = sent_tokenize(re.sub('[\n ]+', ' ', base_text))
    if max_copy_percentage:
       max_copy_count = int(len(org_text_tokenized) * (max_copy_percentage / 100))
    sample_size = max(1, min(max_copy_count, len(org_text_tokenized)))
    indexes_to_insert = random.choice(len(new_text_tokenized), size=sample_size)

    org_text_indexes = random.choice(len(org_text_tokenized), size=sample_size)
    for i in range(sample_size):
        sentence_to_copy = org_text_tokenized[org_text_indexes[i]]
        idx_to_insert = indexes_to_insert[i]
        new_text_tokenized.insert(idx_to_insert, sentence_to_copy)

    2. Near Copy

In [41]:
import numpy
from nltk.tokenize import word_tokenize

def delete_random_words(original_text: str, deletion_percentage: int):
    org_text_tokenized = word_tokenize(original_text)
    deletion_count = max(int(len(org_text_tokenized) * (deletion_percentage / 100)), 1)
    indexes_to_delete = numpy.random.choice(len(org_text_tokenized), size=deletion_count)
    new_text = []
    for i in range(len(org_text_tokenized)):
        if i not in indexes_to_delete:
            new_text.append(org_text_tokenized[i])
    
    return ' '.join(new_text)


def make_near_copy(article_id: str):
    original_article = es.get_article(article_id)
    original_text = get_article_text(article_id, articles_to_plagiarise_dir)
    pruned_title = delete_random_words(original_article['original_title'], 20)
    pruned_abstract = delete_random_words(original_article['original_abstract'], 30)
    pruned_text = delete_random_words(original_text, 40)
    exact_plagiarised_article = make_exact_copy(article_id)
    plagiarised_article = {
        'id': original_article['id'],
        'title': simple_exact_text_copier(pruned_title, base_test_article['original_title'], max_copy_percentage=60),
        'abstract': simple_exact_text_copier(pruned_abstract, base_test_article['original_abstract'], max_copy_percentage=40),
        'text': simple_exact_text_copier(pruned_text, base_test_article_text, max_copy_percentage=10),
        'plagiarism_type': 'near_copy',
    }
    return plagiarised_article

if True:
    import json
    if not os.path.isdir(test_articles_dir):
        os.makedirs(test_articles_dir)
    near_copies_path = f'{test_articles_dir}/near_copies.json'
    with open(near_copies_path, 'w') as file:
        for i in range(near_copies_count):
            original_article_id = metadata['near_copies_ids'][i]
            plagiarised_article = make_near_copy(original_article_id)
            file.write(json.dumps(plagiarised_article) + '\n')

    3. Synonym Substitution

In [42]:
from nltk.corpus import wordnet

def synonym(word: str):
    try: 
        print(f'syn({word}) = {wordnet.synsets(word)[0].lemmas()[0].name()}')
        wordnet.synset(word).w
        return wordnet.synsets(word)[0].lemmas()[0].name()
    except:
        return word

def substitute_random_words(original_text: str, substitution_percentage: int):
    org_text_tokenized = word_tokenize(original_text)
    substitution_count = max(1, int(len(org_text_tokenized) * (substitution_percentage / 100)))
    indexes_to_substitute = numpy.random.choice(len(org_text_tokenized), size=substitution_percentage)
    sbustituted_text_tokenized = org_text_tokenized
    for ind in indexes_to_substitute:
        sbustituted_text_tokenized[ind] = synonym(org_text_tokenized[ind]) 
    
    return ' '.join(sbustituted_text_tokenized)

def near_copy(article_id: str):
    original_article = es.get_article(article_id)
    original_text = get_article_text(article_id, articles_to_plagiarise_dir)
    substituted_title = substitute_random_words(orgiinal_article['original_title'], 20)
    substituted_abstract = substitute_random_words(orgiinal_article['original_abstract'], 20)
    substituted_text = substitute_random_words(original_text, 20)
    plagiarised_article = {
        'id': original_article['id'],
        'title': simple_exact_text_copier(substituted_title, base_test_article['original_title'], max_copy_percentage=60),
        'abstract': simple_exact_text_copier(substituted_abstract, base_test_article['original_abstract'], max_copy_percentage=40),
        'text': simple_exact_text_copier(substituted_text, base_test_article_text, max_copy_percentage=40),
        'plagiarism_type': 'near_copy',
    }
    return plagiarised_article

if True:
    import json
    if not os.path.isdir(test_articles_dir):
        os.makedirs(test_articles_dir)
    substitutes_path = f'{test_articles_dir}/substitutes.json'
    with open(substitutes_path, 'w') as file:
        for i in range(substitutes_count):
            original_article_id = metadata['substitutes_ids'][i]
            plagiarised_article = make_near_copy(original_article_id)
            file.write(json.dumps(plagiarised_article) + '\n')

    4. Plagiarism by paraphrasing

In [12]:


if False:
    import json
    if not os.path.isdir('articles_to_paraphrase/'):
        os.makedirs('articles_to_paraphrase/')
    paraphrases_path = f'articles_to_paraphrase/paraphrases.json'
    with open(paraphrases_path, 'w') as file:
        for i in range(paraphrases_count):
            original_article_id = metadata['paraphrases_ids'][i]
            original_article = es.get_article(original_article_id)
            original_text = get_article_text(original_article_id, articles_to_plagiarise_dir)
            plagiarised_article = {
                'id': original_article['id'],
                'title': original_article['title'],
                'abstract': original_article['original_abstract'],
                'text': original_text,
                'plagiarism_type': 'paraphrasing'
            }
            file.write(json.dumps(plagiarised_article) + '\n')
    
    # the rest is done by hand + online paraphrasing tools

    Not Plagiarised Articles

In [50]:

if True:
    import json
    not_plagiarised_path = f'test_articles/not-plagiarised.json'
    incomplete_articles_path = f'not-plagiarised-articles/not-plagiarised.json'
    with open(not_plagiarised_path, 'w') as file_to_write:
        with open(incomplete_articles_path, 'r') as file_to_read:
            for line in file_to_read.readlines():
                article = json.loads(line)
                original_text = get_article_text(article['id'], articles_to_plagiarise_dir)
                final_article = {
                    'id': article['id'],
                    'title': article['title'],
                    'abstract': article['abstract'],
                    'text': original_text,
                    'plagiarism_type': 'not-plagiarised'
                }
                file_to_write.write(json.dumps(final_article) + '\n')
    

    Declare Detection Methods

In [39]:
from typing import Dict
from nltk.tokenize import sent_tokenize, word_tokenize
from sentence_transformers import util
import nltk
import numpy
import torch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

def find_similarities(sentences_a: str, sentences_b: str):
    a_tokenized = [preprocess(sentence.replace('\n', ' ').strip()) for sentence in sentences_a 
                    if len(word_tokenize(sentence.strip())) > 10] \
                    or [preprocess(sentences_a[0].replace('\n', ' ').strip())]
    b_tokenized = [preprocess(sentence.replace('\n', ' ').strip()) for sentence in sentences_b 
                    if len(word_tokenize(sentence.strip())) > 10] \
                    or [preprocess(sentences_b[0].replace('\n', ' ').strip())]

    a_encoded = numpy.array([model.encode(sentence) for sentence in a_tokenized])
    b_encoded = numpy.array([model.encode(sentence) for sentence in b_tokenized])

    return a_tokenized, b_tokenized, util.pytorch_cos_sim(a_encoded, b_encoded)

def compare_sentences(sentences_a: str, sentences_b: str):
    sentences_a, sentences_b, res = find_similarities(sentences_a, sentences_b)
    flat = res.flatten()
    k = min(len(flat), 10)
    topk = torch.topk(flat, k)
    return topk[0][0] > 0.85


def abstract_query_extractor(article: Dict):
    return [('abstract', article['abstract'])]

def whole_article_query_extractor(article: Dict):
    indexes = numpy.random.choice(len(article['text_tokenized']), size=100)
    queries = [('text', article['text_tokenized'][index]) for index in indexes] + [
        ('title', article['title']),
        ('abstract', article['abstract']),
    ]
    return queries

def abstract_comparator(article_1: Dict, article_2: Dict):
    return [article_1['abstract']], [article_2['abstract']]

def whole_article_comparator(article_1: Dict, article_2: Dict):
    return [article_1['title'], article_1['abstract']] + article_1['text_tokenized'], \
        [article_2['title'], article_2['abstract']] + article_2['text_tokenized']

query_extraction_methods = {
    'abstract': abstract_query_extractor,
    'whole': whole_article_query_extractor,
}

article_comparators = {
    'abstract': abstract_comparator,
    'whole': whole_article_comparator,
}

plagiarism_detection_methods = {
    f'q:{query_extraction_method}_c:{article_comparison_method}': {
        'query_extractor': query_extractor,
        'article_comparator': article_comparator,
    } for query_extraction_method, query_extractor in query_extraction_methods.items()
        for article_comparison_method, article_comparator in article_comparators.items()
}

In [52]:
import json
import csv
import os
import re

correct_guesses = {
    'abstract_abstract': {True: 0, False: 0},
    'abstract_whole': {True: 0, False: 0},
    'whole_abstract': {True: 0, False: 0},
    'whole_whole': {True: 0, False: 0},
}
total = {True: 0, False: 0}


def compute_accuracy(plagiarism_detetion_method: str):
    return (correct_guesses[plagiarism_detection_method][True] + correct_guesses[plagiarism_detection_method][False]) / (total[True] + total[False])

def compute_recall(plagiarism_detection_method: str):
    if total[True]:
        return correct_guesses[plagiarism_detection_method][True] / total[True]
    else:
        return 1

with open('information-retrieval-results.csv', 'w') as ir_results_file:
    ir_csv_writer = csv.writer(ir_results_file)
    ir_csv_writer.writerow(["id","plagiarism_type","abstract_abstract","abstract_whole","whole_abstract","whole_whole"])

    with open('similarity-results.csv', 'w') as similarity_results_file:
        sim_csv_writer = csv.writer(similarity_results_file)
        sim_csv_writer.writerow(["id","plagiarism_type","abstract_abstract","abstract_whole","whole_abstract","whole_whole"])
        for file_name in os.listdir(test_articles_dir):
            # if file_name != 'not-plagiarised.json':
            # if file_name in ['paraphrases.json', 'not-plagiarised.json']:
                # continue
            with open(f'{test_articles_dir}/{file_name}', 'r') as file:
                correct_guesses = {
                    'abstract_abstract': {True: 0, False: 0},
                    'abstract_whole': {True: 0, False: 0},
                    'whole_abstract': {True: 0, False: 0},
                    'whole_whole': {True: 0, False: 0},
                }
                total = {True: 0, False: 0}

                for i, line in enumerate(file.readlines()):
                    if i == 4:
                        break
                    test_article = json.loads(line)
                    
                    test_article['text_tokenized'] = sent_tokenize(re.sub('[ \n]+', ' ', test_article['text']))

                    # print('the first few sentences of test article are: ')
                    # for i in range(min(len(test_article['text_tokenized']), 4)):
                    #     print(test_article['text_tokenized'][i], end='\n-------------\n')

                    # print(print_divider)


                    sim_result_row = {'id': test_article['id'], 'plagiarism_type': test_article['plagiarism_type']}
                    ir_result_row = {'id': test_article['id'], 'plagiarism_type': test_article['plagiarism_type']}
                    is_article_plagiarised = test_article['plagiarism_type'] != 'not-plagiarised'
                    total[is_article_plagiarised] += 1

                    for query_extraction_method, query_extractor in query_extraction_methods.items():
                        for article_comparison_method, article_comparator in article_comparators.items(): 
                            plagiarism_detection_method = query_extraction_method + '_' + article_comparison_method
                            # if plagiarism_detection_method != 'whole_whole':
                                # continue

                            queries = query_extractor(test_article)
                            elasticsearch_results = es.search(queries=queries, n=1)

                            for i, es_res in enumerate(elasticsearch_results['hits']['hits']):
                                candidate_article = es_res['_source']

                                candidate_article_text_raw = get_article_text(candidate_article['id'], articles_to_plagiarise_dir)
                                candidate_article_text = re.sub('[ \n]+', ' ', candidate_article_text_raw)

                                candidate_article.update({
                                    'text': candidate_article_text,
                                    'text_tokenized': sent_tokenize(candidate_article_text),
                                })
                                test_article_sentences, candidate_article_sentencess = article_comparator(test_article, candidate_article)
                                plagiarism_detected = bool(compare_sentences(test_article_sentences, candidate_article_sentencess))
                                sim_result_row[plagiarism_detection_method] = plagiarism_detected
                                ir_result_row[plagiarism_detection_method] = candidate_article['id'] == test_article['id']
                                if plagiarism_detected == is_article_plagiarised:
                                    correct_guesses[plagiarism_detection_method][plagiarism_detected] += 1
                    sim_csv_writer.writerow(sim_result_row.values())
                    ir_csv_writer.writerow(ir_result_row.values())
            for plagiarism_detection_method in correct_guesses:
                print(f'accuracy and recall of {plagiarism_detection_method} on {file_name.split(".")[0]} are {compute_accuracy(plagiarism_detection_method)} and {compute_recall(plagiarism_detection_method)}')


accuracy and recall of abstract_abstract on paraphrases are 0.5 and 0.5
accuracy and recall of abstract_whole on paraphrases are 1.0 and 1.0
accuracy and recall of whole_abstract on paraphrases are 0.5 and 0.5
accuracy and recall of whole_whole on paraphrases are 1.0 and 1.0
accuracy and recall of abstract_abstract on exact_copies are 0.0 and 0.0
accuracy and recall of abstract_whole on exact_copies are 0.75 and 0.75
accuracy and recall of whole_abstract on exact_copies are 0.0 and 0.0
accuracy and recall of whole_whole on exact_copies are 1.0 and 1.0
accuracy and recall of abstract_abstract on substitutes are 0.0 and 0.0
accuracy and recall of abstract_whole on substitutes are 0.75 and 0.75
accuracy and recall of whole_abstract on substitutes are 0.0 and 0.0
accuracy and recall of whole_whole on substitutes are 0.75 and 0.75
accuracy and recall of abstract_abstract on not-plagiarised are 1.0 and 1
accuracy and recall of abstract_whole on not-plagiarised are 1.0 and 1
accuracy and reca

In [10]:
# import torch
# print(len(sent_tokenize(s1))) # 315
# print(res.shape) torch.Size([315, 231])

    # break
    # print('*' * 20 + f'\nthe following sentences have a similarity of {topk[0][i]}:')
    # a_index, b_index = topk[1][i] // len(sentences_a), topk[1][i] % len(sentences_a)
    # a_sentence, b_sentence = sentences_a[a_index].replace('\n', ' '), sentences_b[b_index].replace('\n', ' ')
    # print(f"{a_sentence}\n**************\n{b_sentence}\n")
# max_similarity_per_sentence = torch.amax(res, axis=1)
# sorted_similarities = torch.sort(max_similarity_per_sentence)
# print(sorted_similarities)
# print(sorted_similarities[0][-10:], '\n', sorted_similarities[1][-10:])

In [11]:
from dask import dataframe as dask_dataframe
df = dask_dataframe.read_json(arxiv_complete_path, blocksize=25e6)

Preprocess Data

In [2]:
from IPython.utils import io
with io.capture_output() as captured:
    ! pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

In [12]:
# df['processed_abstract'] = df['abstract'].apply(preprocess)

ModuleNotFoundError: No module named 'en_core_sci_lg'